In [1]:
import pyomo.environ as pyo
from pyomo.dae import ContinuousSet, DerivativeVar

In [2]:
tf = 63
CA0 = 0.3335  # initial
CB0 = 0
k = 0.1 
n = 1.54

m = pyo.ConcreteModel()

########################################################################
# Define model variables
# time
m.t = ContinuousSet(bounds=(0, tf))

# concentration of A and B
m.CA = pyo.Var(m.t, domain = pyo.NonNegativeReals)#, initialize= 1e-6)
m.CB = pyo.Var(m.t, domain = pyo.NonNegativeReals)#, initialize= 1e-6)

m.k = pyo.Param(initialize = k)
m.n = pyo.Param(initialize = n)

# Differential variable
m.dCAdt = DerivativeVar(m.CA, wrt = m.t)
# m.dCBdt = DerivativeVar(m.CB, wrt = m.t)

# Expression for rate
@m.Constraint(m.t)
def CA_rxn_rate(m, t):
    return m.dCAdt[t] == - m.k * (m.CA[t]+1e-12)**m.n

m.CA[0].fix(CA0)

discr = pyo.TransformationFactory('dae.collocation')
discr.apply_to(m, nfe = 20, ncp=10)

solver = pyo.SolverFactory("ipopt")
solver.solve(m, tee=True)

# print
ca = [pyo.value(m.CA[t]) for t in m.t]
print(ca)

Ipopt 3.14.17: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     2791
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      200

Total number of variables............................:      401
                     variables with only lower bounds:      200
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      401
Total number